<a href="https://colab.research.google.com/github/Kim-TaeKyoung/one/blob/main/0_Alphapose_with_Heuristic_applied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pyyaml==5.2
! pip install scipy==1.1.0
! pip install torch==1.2.0 torchvision==0.4.0
! pip install pillow==6.2.2
import torch
print(torch.__version__)
import yaml, scipy
import os, sys
print(yaml.__version__)
print(scipy.__version__)

In [ ]:
!apt install ffmpeg

%env SHARED_DIR = /content/drive/MyDrive/Colab/
%env ALPHAPOSE_DIR = /content/drive/MyDrive/Colab/AlphaPose
sys.path.append(os.environ['ALPHAPOSE_DIR'])
#!git clone https://github.com/MVIG-SJTU/AlphaPose.git $ALPHAPOSE_DIR

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
env: SHARED_DIR=/content/drive/MyDrive/Colab/
env: ALPHAPOSE_DIR=/content/drive/MyDrive/Colab/AlphaPose


In [ ]:
!pwd

/content


In [ ]:
!python -m pip install cython
!apt-get install libyaml-dev

print(os.getcwd())
os.chdir(os.environ['ALPHAPOSE_DIR'])
!python setup.py build develop

In [ ]:
sys.path.append(os.environ['ALPHAPOSE_DIR'])
os.environ['CSV_DIR'] = os.environ['SHARED_DIR'] + 'FieldTestCSV/AllCSVs/'
os.environ['VIDEO_DIR'] = os.environ['SHARED_DIR'] + 'frames/'

!echo $CSV_DIR
!echo $VIDEO_DIR

/content/drive/MyDrive/Colab/FieldTestCSV/AllCSVs/
/content/drive/MyDrive/Colab/frames/


In [ ]:
import os
os.chdir(os.environ['VIDEO_DIR'])

!find -type d -maxdepth 1

In [ ]:
#@title Use vis_fast if csv is only needed one
%%writefile test.sh
cd $ALPHAPOSE_DIR
echo "Cycle for ${TARGET_DATE}"
find $TARGET_DATE -type f -name '*.mp4' | while read -r FILENAME; do
  echo "Alphaposing now... : ${FILENAME}"
  CSV_FILE=$(echo "${FILENAME}" | sed "s/mp4/csv/g")
  python3 scripts/demo_inference.py --detector tracker \
                                    --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml \
                                    --checkpoint pretrained_models/fast_res50_256x192.pth \
                                    --video $FILENAME \
                                    --vis_fast
                                    #--save_video
  echo "Saving Results to ${CSV_FILE}"
  mv examples/res/_heuristic $CSV_FILE
done

Overwriting test.sh


In [ ]:
#@title Subscript the output array to ranging out the target date
import subprocess
import os

os.chdir(os.environ['VIDEO_DIR'])
cmd = subprocess.Popen('find -type d -maxdepth 1', shell=True, stdout=subprocess.PIPE)
os.chdir(os.environ['ALPHAPOSE_DIR'])

output = list(cmd.stdout)

for line in output[7:8]:
  os.environ['TARGET_DATE'] = os.path.basename(line.decode().rstrip('\n'))
  os.environ['TARGET_DATE'] = os.environ['VIDEO_DIR'] + os.environ['TARGET_DATE']

  ap = subprocess.Popen('sh test.sh', shell=True, stdout=subprocess.PIPE)
  for line in ap.stdout:
    print(line)

In [ ]:
#@title For Debugging purposes
%env TARGET_DATE=2021_10_14
os.environ['TARGET_DATE'] = os.environ['VIDEO_DIR'] + os.environ['TARGET_DATE']
!echo $TARGET_DATE
os.chdir(os.environ['ALPHAPOSE_DIR'])
!pwd
!sh ${ALPHAPOSE_DIR}test.sh

In [ ]:
'''
#@title In case of pretrained weights lost, uncomment this and run.
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

! mkdir ${ALPHAPOSE_DIR}detector/yolo/data
file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(os.environ['ALPHAPOSE_DIR'] + 'detector/yolo/data/yolov3-spp.weights')

! mkdir ${ALPHAPOSE_DIR}detector/tracker/data
file_id = '1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(os.environ['ALPHAPOSE_DIR'] + 'detector/tracker/data/JDE-1088x608-uncertainty')

os.rename(os.environ['ALPHAPOSE_DIR'] + 'detector/tracker/data/JDE-1088x608-uncertainty',\
          os.environ['ALPHAPOSE_DIR'] + 'detector/tracker/data/jde.1088x608.uncertainty.pt')

file_id = '1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(os.environ['ALPHAPOSE_DIR'] + 'pretrained_models/fast_res50_256x192.pth')
'''